In [1]:
import pandas as pd
import preprocessor_improved as prep_improved
import preprocessor_baseline as prep_baseline
from random_forest_regressor import random_forest_regressor
from linear_regression import linear_regression
from gbm import xg_boost
from sklearn.model_selection import train_test_split
import warnings
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn import metrics
warnings.filterwarnings('ignore')



# Load the datasets
singapore_df = pd.read_csv('datasets/singapore_listings.csv')
ny_df = pd.read_csv('datasets/newyorkcity_listings.csv')
madrid_df = pd.read_csv('datasets/madrid_listings.csv')

C:\Users\Devon Murray\AppData\Roaming\Python\Python311\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:

# Preprocess the data
singapore_preproccessor, singapore_X, singapore_y = prep_baseline.preprocess_data(singapore_df)
singapore_X_transformed = singapore_preproccessor.fit_transform(singapore_X)
# ny_preproccessor, ny_X, ny_y = prep_baseline.preprocess_data(ny_df)
# madrid_preproccessor, madrid_X, madrid_y = prep_baseline.preprocess_data(madrid_df)


# Split the data into training and testing sets
singapore_X_train, singapore_X_test, singapore_y_train, singapore_y_test = train_test_split(singapore_X_transformed, singapore_y, test_size=0.3, random_state=42)

model = RandomForestRegressor()

n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1000, num = 20)]
max_features = ['log2', 'sqrt']
max_depth = [int(x) for x in np.linspace(10, 150, num = 11)]
min_samples_split = [2, 5, 10, 20]
min_samples_leaf = [1, 2, 4, 10, 20]
bootstrap = [True, False]

parametrs = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

rnd_search_cv = RandomizedSearchCV(model, parametrs, cv=5)
rnd_search_cv.fit(singapore_X_train, singapore_y_train)

RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(),
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 24, 38, 52, 66, 80,
                                                      94, 108, 122, 136, 150],
                                        'max_features': ['log2', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4, 10, 20],
                                        'min_samples_split': [2, 5, 10, 20],
                                        'n_estimators': [100, 147, 194, 242,
                                                         289, 336, 384, 431,
                                                         478, 526, 573, 621,
                                                         668, 715, 763, 810,
                                                         857, 905, 952, 1000]})

In [3]:
predictions = rnd_search_cv.predict(singapore_X_test)

#Mean Absolute Error 
print('MAE:', metrics.mean_absolute_error(singapore_y_test, predictions))

#Mean Squared Error
print('MSE:', metrics.mean_squared_error(singapore_y_test, predictions))


#Root Mean Squared Error
print('RMSE:', np.sqrt(metrics.mean_squared_error(singapore_y_test, predictions)))

#R2
print('R2:', metrics.r2_score(singapore_y_test, predictions))

MAE: 75.56751131152274
MSE: 57299.51008290595
RMSE: 239.3731607405182
R2: 0.1073899107266103


In [6]:
pd.Series(rnd_search_cv.best_estimator_.feature_importances_, index=singapore_df.columns).sort_values(ascending=False)

ValueError: Length of values (58) does not match length of index (20)